In [1]:
import pandas as pd
import os
from copy import deepcopy
import numpy as np
from math import isclose

os.chdir("..")
from adaptive_multigrid import give_data_individual_names, find_best_individual_name, build_individual_dict_from_param_grid, update_param_grid
from psuu import scenario_configs, build_next_param_config_code
from model.config.params import *



sweep = "gateway_viability_sweep_ag1_"
sweep_family = sweep[:sweep.index("_ag")+3]
kpis = pd.read_csv("simulation_data/{}.csv".format(sweep), index_col=0)

exec("param_config = {}".format(sweep))
next_name = sweep[:-2] + str(int(sweep[-2]) + 1) + "_"
fitness_func = scenario_configs[sweep_family]['fitness_function']
variable_params = scenario_configs[sweep_family]['variable_params']
control_params = scenario_configs[sweep_family]['control_params']
assert False
params_to_evolve = {}
for x in variable_params:
    params_to_evolve[x] = param_config[x]

params_for_setup_only = {}
for x in control_params:
    params_for_setup_only[x] = param_config[x]

initial_param_grid = params_to_evolve
other_params_to_sweep = params_for_setup_only

new_param_grid = deepcopy(initial_param_grid)
new_best_name = ""
new_best_fitness = -np.inf
times_repeat_best_name = 0
times_repeat_best_fitness = 0

old_param_grid = deepcopy(new_param_grid)
old_best_name = new_best_name
old_best_fitness = new_best_fitness

individual_dict = build_individual_dict_from_param_grid(old_param_grid)
        
        

data = kpis
data = give_data_individual_names(data)

new_best_name, new_best_val = find_best_individual_name(
            data, fitness_func=fitness_func, name_biased_towards=old_best_name
        )

assert not (
            new_best_name is None
        ), "The best individual should be somewhere in this dictionary."
print(new_best_name)

new_best_individual = individual_dict[new_best_name]

if new_best_name == old_best_name:
    times_repeat_best_name = times_repeat_best_name + 1
else:
    times_repeat_best_name = 0

if isclose(new_best_fitness, old_best_fitness):
    times_repeat_best_fitness = times_repeat_best_fitness + 1
else:
    times_repeat_best_fitness = 0
    
new_param_grid = update_param_grid(
                old_param_grid=old_param_grid, best_individual=new_best_individual
            )

/Users/seanmcowen/opt/anaconda3/envs/BlockScience/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AssertionError: 

In [44]:
thresholds = {"a1": .1,
             "a2": .9,
             "s1": 750*1e6,
             "s2": .9,
             "t1": 7500*1e6,
             "t2": .9}
grouping = kpis.groupby(["param_" + x for x in variable_params])

In [52]:
# Get average cost
thresh1 = grouping.apply(lambda x: x["KPI 14"].mean())
# Check if bounded between
thresh1 = (thresh1 > thresholds["a1"]) & (thresh1 < thresholds["a2"])

# Check average monte carlo success for servicer NPV
thresh2 = grouping.apply(lambda x: (x["KPI 1"] > thresholds["s1"]).mean())
# Check number of successful parameter constellations
thresh2 = thresh2 > thresholds["s2"]

# Check average monte carlo success for gateway NPV
thresh3 = grouping.apply(lambda x: (x["KPI 3"] > thresholds["t1"]).mean())
# Check number of successful parameter constellations
thresh3 = thresh3 > thresholds["t2"]

thresh = pd.concat([thresh1, thresh2, thresh3], axis=1)
thresh.columns = ["Threshold Servicer Capital Costs", "Threshold Servicer NPV", "Threshold Gateway NPV"]
print(thresh.mean())

Threshold Servicer Capital Costs    0.0
Threshold Servicer NPV              0.0
Threshold Gateway NPV               0.0
dtype: float64


In [34]:
kpis["KPI 1"].max() > 750*1e6

True

In [25]:
scores["Servicer Capital Costs"] = ((scores["KPI 14"] > thresholds["a1"]) & (scores["KPI 14"] < thresholds["a2"]))


In [26]:
scores["Servicer Capital Costs"]

0       False
1       False
2       False
3       False
4       False
        ...  
1435    False
1436    False
1437    False
1438    False
1439    False
Name: Servicer Capital Costs, Length: 1440, dtype: bool

In [16]:
list(scores.columns)

['Net Minting Rate',
 'KPI C',
 'Experiment Name',
 'State Set',
 'Params Set',
 'timestep',
 'param_minimum_stake_servicer',
 'param_minimum_stake_period_servicer',
 'param_minimum_pause_time',
 'param_max_chains_servicer',
 'param_relays_to_tokens_multiplier',
 'param_slash_fraction_downtime',
 'param_downtime_jail_duration',
 'param_minimum_servicers_per_session',
 'param_maximum_servicers_per_session',
 'param_application_fee_per_relay',
 'param_minimum_application_stake',
 'param_app_burn_per_session',
 'param_app_burn_per_relay',
 'param_block_proposer_allocation',
 'param_dao_allocation',
 'param_servicer_allocation',
 'param_stake_per_app_delegation',
 'param_gateway_fee_per_relay',
 'param_gateway_minimum_stake',
 'param_session_token_bucket_coefficient',
 'param_dao_fee_percentage',
 'param_validator_fee_percentage',
 'param_transaction_fee',
 'param_min_bootstrap_gateway_fee_per_relay',
 'param_maturity_relay_charge',
 'param_gateway_bootstrap_unwind_start',
 'param_gateway_

In [9]:
grouping.apply(lambda x: )

In [2]:

build_next_param_config_code(
    next_name, new_param_grid, control_params, param_config
)

Add the following to model/config/params.py:

gateway_viability_sweep_ag6_ = build_params("Base")
gateway_viability_sweep_ag6_["session_token_bucket_coefficient"] = [189.0625, 200.78125]
gateway_viability_sweep_ag6_["gateway_fee_per_relay"] = [21.875, 22.1875]
gateway_viability_sweep_ag6_["application_fee_per_relay"] = [25.625, 25.9375]
gateway_viability_sweep_ag6_["gateway_minimum_stake"] = [137500000000.0, 140625000000.0]
gateway_viability_sweep_ag6_["minimum_application_stake"] = [187500000000.0, 190625000000.0]
gateway_viability_sweep_ag6_["application_max_number"] = [5, 20, 100]
gateway_viability_sweep_ag6_["relays_per_session_gamma_distribution_scale"] = [100000, 300000, 900000]
create_sweep("gateway_viability_sweep_ag6_",gateway_viability_sweep_ag6_,config_option_map_sweep)


Add the following to model/config/experiment.py:

for i in range(1, 289):
    experimental_setups["gateway_viability_sweep_ag6_{}".format(i)] = {
        "config_option_state": "Base",
        "config_optio

In [4]:
list(kpis.columns)

['Net Minting Rate',
 'KPI C',
 'Experiment Name',
 'State Set',
 'Params Set',
 'timestep',
 'param_minimum_stake_servicer',
 'param_minimum_stake_period_servicer',
 'param_minimum_pause_time',
 'param_max_chains_servicer',
 'param_relays_to_tokens_multiplier',
 'param_slash_fraction_downtime',
 'param_downtime_jail_duration',
 'param_minimum_servicers_per_session',
 'param_maximum_servicers_per_session',
 'param_application_fee_per_relay',
 'param_minimum_application_stake',
 'param_app_burn_per_session',
 'param_app_burn_per_relay',
 'param_block_proposer_allocation',
 'param_dao_allocation',
 'param_servicer_allocation',
 'param_stake_per_app_delegation',
 'param_gateway_fee_per_relay',
 'param_gateway_minimum_stake',
 'param_session_token_bucket_coefficient',
 'param_dao_fee_percentage',
 'param_validator_fee_percentage',
 'param_transaction_fee',
 'param_min_bootstrap_gateway_fee_per_relay',
 'param_maturity_relay_charge',
 'param_gateway_bootstrap_unwind_start',
 'param_gateway_